In [9]:
import tensorflow as tf
import os
import numpy as np
from copy import deepcopy
import sys

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
import functools
from collections import Counter

In [10]:
# create record files

dirs = 'record/noise_0.8_iteration_5_cleansize_50_add_1000/test1/'
if not os.path.exists(dirs):
    os.makedirs(dirs)

file_setting = open(dirs + 'file_setting.txt', 'a+')
file_additional_data = open(dirs + 'file_additional_data.txt', 'a+')
file_teacher = open(dirs + 'file_teacher.txt', 'a+')
file_student = open(dirs + 'file_student.txt', 'a+')
file_benchmark = open(dirs + 'file_benchmark.txt', 'a+')

In [11]:
# load data from CIFAR10
def load_data(clean_data_size):
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

    # transform labels to one-hot vectors
    y_train = tf.contrib.keras.utils.to_categorical(y_train, 10)
    y_test = tf.contrib.keras.utils.to_categorical(y_test, 10)
    
    clean_index = []
    for label in range(10):
        positive_index = list(np.where(y_train[:, label] == 1)[0])        
        clean_index = np.append(clean_index, np.random.choice(positive_index, clean_data_size, replace=False)).astype(int)
    
    x_clean = x_train[clean_index]
    y_clean = y_train[clean_index]
    x_train = np.delete(x_train, clean_index, axis=0)
    y_train = np.delete(y_train, clean_index, axis=0)
    return x_train, y_train, x_test, y_test, x_clean, y_clean


def generate_noise_labels(y_train, noise_level):
    num_noise = int(noise_level * y_train.shape[0])
    noise_index = np.random.choice(y_train.shape[0], num_noise, replace=False)
    label_slice = np.argmax(y_train[noise_index], axis=1)
    new_label = np.random.randint(low=0, high=10, size=num_noise)
    while sum(label_slice == new_label) > 0:
        n = sum(label_slice == new_label)
        new_label[label_slice == new_label] = np.random.randint(low=0, high=10, size=n)
    y_train[noise_index] = tf.contrib.keras.utils.to_categorical(new_label, 10)
    return y_train


In [12]:
noise_level = 0.8
clean_data_size = 50
seed = 10

np.random.seed(seed)
tf.set_random_seed(seed)

x_train, y_train, x_test, y_test, x_clean, y_clean = load_data(clean_data_size)
y_train_orig = deepcopy(y_train)
y_train = generate_noise_labels(y_train, noise_level)

file_setting.write('noise level: ' + str(noise_level) + '\n')
file_setting.write('clean data size for each class: ' + str(clean_data_size) + '\n')
file_setting.close()



In [13]:
def create_model(architecture, num_classes, learning_rate=0.001, dropout=0.5):
    model = Sequential()
    for layer_index in range(len(architecture)):
        layer = architecture[layer_index]
        if len(layer) == 3:
            if layer_index == 0:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), input_shape=(32, 32, 3),
                                 kernel_initializer='glorot_normal', activation='relu', padding='same'))
            else:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), kernel_initializer='glorot_normal',
                                 activation='relu', padding='same'))
            if layer_index < 3:
                model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        elif len(layer) == 1:
            if len(architecture[layer_index - 1]) == 3:
                model.add(Flatten())
            model.add(Dense(layer[0], activation='relu', kernel_initializer='glorot_normal'))
        else:
            print('Invalid architecture /(ㄒoㄒ)/~~')
    model.add(Dropout(dropout))
    if num_classes > 2:
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    elif num_classes == 2:
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)   
    return model

In [17]:
# generate 10 binary classifier
binary_classifier_list = []
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [256]]
for label in range(10):
    model = create_model(architecture, num_classes=2)
    binary_classifier_list.append(model)

file_setting = open(dirs + 'file_setting.txt', 'a+')
file_setting.write('*'*10 + 'architecture of binary classifier' + '*'*10 + '\n')
orig_stdout = sys.stdout
sys.stdout = file_setting
print(model.summary())
sys.stdout = orig_stdout
file_setting.close()
    

In [18]:
# use the idea of PU learning to augment positive data
epochs = 5
additional_data_index = [[] for i in range(10)]
for epoch in range(epochs):
    for label in range(10):
        print('epoch', epoch, 'label', label)
        positive_index = list(np.where(y_clean[:, label] == 1)[0])
        x = x_clean[positive_index]
        x = np.concatenate((x, x_train[additional_data_index[label]]), axis=0)
        n_p = len(x)
        n_n = min(400, n_p)
        negative_index = list(np.where(y_clean[:, label] != 1)[0])
        negative_index = np.random.choice(negative_index, n_n, replace=False)
        x = np.concatenate((x, x_clean[negative_index]), axis=0)
        y = [1] * n_p + [0] * n_n
        classifier = binary_classifier_list[label]
        classifier.fit(x, y, batch_size=32, epochs=20, shuffle=True)
        pred_train = classifier.predict(x_train)
        candidate_index = np.where(pred_train > 0.98)[0]
        if len(candidate_index) < 1000:
            additional_data_index[label] = list(candidate_index)
        else:
            additional_data_index[label] = np.argsort(-pred_train, axis=0)[0:1000].reshape(-1)
    
    # estimate additional clean data
    precision_additional_data = []
    number_additional_data = []
    for label in range(10):
        index = additional_data_index[label]
        true_positive_index = list(np.where(y_train_orig[:, label] != 1)[0])
        TP = len(list(set(index) & set(true_positive_index)))
        if len(index) == 0:
            precision_additional_data.append(0)
        else:
            precision_additional_data.append(TP/len(index))  
        number_additional_data.append(len(index))
    print(precision_additional_data)
    print(number_additional_data)

    file_additional_data.write('iteraion ' + str(epoch) + ', precision of additional data for each class' + '\n')
    file_additional_data.write(str(precision_additional_data) + '\n')
    file_additional_data.write('iteraion ' + str(epoch) + ', number of additional data for each class' + '\n')
    file_additional_data.write(str(number_additional_data) + '\n')
    file_additional_data.flush()
file_additional_data.close()

epoch 0 label 0
Epoch 1/20
100/100 [==============================] - 2s 15ms/step - loss: 0.2707 - acc: 0.4900
Epoch 2/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2472 - acc: 0.5000
Epoch 3/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2455 - acc: 0.4900
Epoch 4/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2489 - acc: 0.5000
Epoch 5/20
100/100 [==============================] - 0s 3ms/step - loss: 0.2380 - acc: 0.5000
Epoch 6/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2413 - acc: 0.5000
Epoch 7/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2326 - acc: 0.5400
Epoch 8/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2341 - acc: 0.6000
Epoch 9/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2198 - acc: 0.6800
Epoch 10/20
100/100 [==============================] - 0s 2ms/step - loss: 0.1996 - acc: 0.7200
Epoch 11/20
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.2486 - acc: 0.5100
Epoch 6/20
100/100 [==============================] - 0s 812us/step - loss: 0.2522 - acc: 0.5000
Epoch 7/20
100/100 [==============================] - 0s 868us/step - loss: 0.2476 - acc: 0.4900
Epoch 8/20
100/100 [==============================] - 0s 926us/step - loss: 0.2481 - acc: 0.5400
Epoch 9/20
100/100 [==============================] - 0s 866us/step - loss: 0.2468 - acc: 0.5300
Epoch 10/20
100/100 [==============================] - 0s 926us/step - loss: 0.2439 - acc: 0.5700
Epoch 11/20
100/100 [==============================] - 0s 885us/step - loss: 0.2473 - acc: 0.5300
Epoch 12/20
100/100 [==============================] - 0s 917us/step - loss: 0.2438 - acc: 0.6100
Epoch 13/20
100/100 [==============================] - 0s 830us/step - loss: 0.2440 - acc: 0.6100
Epoch 14/20
100/100 [==============================] - 0s 968us/step - loss: 0.2388 - acc: 0.6100
Epoch 15/20
100/100 [=================

100/100 [==============================] - 0s 845us/step - loss: 0.1171 - acc: 0.8700
Epoch 13/20
100/100 [==============================] - 0s 809us/step - loss: 0.1085 - acc: 0.8400
Epoch 14/20
100/100 [==============================] - 0s 832us/step - loss: 0.1289 - acc: 0.8200
Epoch 15/20
100/100 [==============================] - 0s 791us/step - loss: 0.1163 - acc: 0.8500
Epoch 16/20
100/100 [==============================] - 0s 797us/step - loss: 0.1146 - acc: 0.8600
Epoch 17/20
100/100 [==============================] - 0s 785us/step - loss: 0.0853 - acc: 0.9300
Epoch 18/20
100/100 [==============================] - 0s 780us/step - loss: 0.0972 - acc: 0.8600
Epoch 19/20
100/100 [==============================] - 0s 763us/step - loss: 0.0748 - acc: 0.9100
Epoch 20/20
100/100 [==============================] - 0s 775us/step - loss: 0.1072 - acc: 0.8500
epoch 1 label 3
Epoch 1/20
1141/1141 [==============================] - 1s 872us/step - loss: 0.0597 - acc: 0.9220
Epoch 2/20
1141

984/984 [==============================] - 2s 2ms/step - loss: 0.0078 - acc: 0.9919
Epoch 17/20
984/984 [==============================] - 1s 1ms/step - loss: 0.0068 - acc: 0.9959
Epoch 18/20
984/984 [==============================] - 1s 862us/step - loss: 0.0057 - acc: 0.9939
Epoch 19/20
984/984 [==============================] - 1s 789us/step - loss: 0.0080 - acc: 0.9909
Epoch 20/20
984/984 [==============================] - 1s 821us/step - loss: 0.0088 - acc: 0.9888
epoch 2 label 1
Epoch 1/20
864/864 [==============================] - 1s 836us/step - loss: 0.0353 - acc: 0.9595
Epoch 2/20
864/864 [==============================] - 1s 773us/step - loss: 0.0236 - acc: 0.9745
Epoch 3/20
864/864 [==============================] - 1s 770us/step - loss: 0.0180 - acc: 0.9780
Epoch 4/20
864/864 [==============================] - 1s 842us/step - loss: 0.0164 - acc: 0.9780
Epoch 5/20
864/864 [==============================] - 1s 2ms/step - loss: 0.0134 - acc: 0.9838
Epoch 6/20
864/864 [=======

1450/1450 [==============================] - 2s 2ms/step - loss: 0.0119 - acc: 0.9876
Epoch 2/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0109 - acc: 0.9890
Epoch 3/20
1450/1450 [==============================] - 1s 752us/step - loss: 0.0109 - acc: 0.9890
Epoch 4/20
1450/1450 [==============================] - 1s 755us/step - loss: 0.0102 - acc: 0.9890
Epoch 5/20
1450/1450 [==============================] - 1s 751us/step - loss: 0.0085 - acc: 0.9903
Epoch 6/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0085 - acc: 0.9917
Epoch 7/20
1450/1450 [==============================] - 3s 2ms/step - loss: 0.0123 - acc: 0.9848
Epoch 8/20
1450/1450 [==============================] - 3s 2ms/step - loss: 0.0117 - acc: 0.9862
Epoch 9/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0084 - acc: 0.9910
Epoch 10/20
1450/1450 [==============================] - 1s 775us/step - loss: 0.0084 - acc: 0.9903
Epoch 11/20
1450/1450 [=========

1450/1450 [==============================] - 1s 754us/step - loss: 0.0021 - acc: 0.9979
Epoch 4/20
1450/1450 [==============================] - 1s 800us/step - loss: 0.0021 - acc: 0.9979
Epoch 5/20
1450/1450 [==============================] - 1s 780us/step - loss: 0.0021 - acc: 0.9979
Epoch 6/20
1450/1450 [==============================] - 1s 751us/step - loss: 0.0021 - acc: 0.9979
Epoch 7/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0021 - acc: 0.9979
Epoch 8/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0020 - acc: 0.9979
Epoch 9/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0015 - acc: 0.9986
Epoch 10/20
1450/1450 [==============================] - 2s 2ms/step - loss: 0.0015 - acc: 0.9986
Epoch 11/20
1450/1450 [==============================] - 1s 781us/step - loss: 0.0051 - acc: 0.9931
Epoch 12/20
1450/1450 [==============================] - 1s 748us/step - loss: 0.0049 - acc: 0.9945
Epoch 13/20
1450/1450 [===

1450/1450 [==============================] - 2s 1ms/step - loss: 0.0090 - acc: 0.9897
Epoch 6/20
1450/1450 [==============================] - 1s 805us/step - loss: 0.0064 - acc: 0.9931
Epoch 7/20
1450/1450 [==============================] - 1s 806us/step - loss: 0.0056 - acc: 0.9938
Epoch 8/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0085 - acc: 0.9897
Epoch 9/20
1450/1450 [==============================] - 3s 2ms/step - loss: 0.0037 - acc: 0.9959
Epoch 10/20
1450/1450 [==============================] - 3s 2ms/step - loss: 0.0037 - acc: 0.9959
Epoch 11/20
1450/1450 [==============================] - 1s 799us/step - loss: 0.0022 - acc: 0.9979
Epoch 12/20
1450/1450 [==============================] - 1s 761us/step - loss: 0.0050 - acc: 0.9945
Epoch 13/20
1450/1450 [==============================] - 1s 763us/step - loss: 0.0050 - acc: 0.9938
Epoch 14/20
1450/1450 [==============================] - 1s 1ms/step - loss: 0.0014 - acc: 0.9986
Epoch 15/20
1450/1450 [===

In [19]:
# get additional data and train teacher model
x_add = deepcopy(x_clean)
y_add = deepcopy(y_clean)
for label in range(10):
    index = additional_data_index[label]
    x_add = np.concatenate((x_add, x_train[index]), axis=0)
    y_add = np.concatenate((y_add, tf.contrib.keras.utils.to_categorical([label]*len(index), 10)))

architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
teacher_model = create_model(architecture, num_classes=10)

file_setting = open(dirs + 'file_setting.txt', 'a+')
file_setting.write('*'*10 + 'architecture of teacher classifier' + '*'*10 + '\n')
orig_stdout = sys.stdout
sys.stdout = file_setting
print(teacher_model.summary())
sys.stdout = orig_stdout
file_setting.close()

History_teacher = teacher_model.fit(x_add, y_add, validation_data=(x_test, y_test), batch_size=64, epochs=100, shuffle=True)

file_teacher.write('training accuracy' + '\n')
file_teacher.write(str(History_teacher.history['acc']) + '\n')
file_teacher.write('test accuracy' + '\n')
file_teacher.write(str(History_teacher.history['val_acc']) + '\n')
file_teacher.close()


Train on 10500 samples, validate on 10000 samples
Epoch 1/100
10500/10500 [==============================] - 46s 4ms/step - loss: 1.4457 - acc: 0.4734 - val_loss: 2.1814 - val_acc: 0.3003
Epoch 2/100
10500/10500 [==============================] - 45s 4ms/step - loss: 0.9998 - acc: 0.6364 - val_loss: 2.1398 - val_acc: 0.3069
Epoch 3/100
10500/10500 [==============================] - 46s 4ms/step - loss: 0.8857 - acc: 0.6749 - val_loss: 2.2491 - val_acc: 0.3124
Epoch 4/100
10500/10500 [==============================] - 51s 5ms/step - loss: 0.8103 - acc: 0.6940 - val_loss: 2.3408 - val_acc: 0.3172
Epoch 5/100
10500/10500 [==============================] - 47s 4ms/step - loss: 0.7548 - acc: 0.7166 - val_loss: 2.3124 - val_acc: 0.3217
Epoch 6/100
10500/10500 [==============================] - 47s 4ms/step - loss: 0.7112 - acc: 0.7271 - val_loss: 2.3886 - val_acc: 0.3240
Epoch 7/100
10500/10500 [==============================] - 49s 5ms/step - loss: 0.6808 - acc: 0.7316 - val_loss: 2.4563 - 

In [24]:
# calculate lambda
def get_precision(x, y, model):
    mean_precision = []
    prediction = model.predict(x)
    y_pred = prediction.argmax(axis=-1)
    for label in range(10):        
        pred_positive_index = list(np.where(y_pred == label)[0])
        true_positive_index = list(np.where(y[:, label] == 1)[0])
        TP = len(list(set(pred_positive_index) & set(true_positive_index)))
        print(label, len(pred_positive_index), len)
        mean_precision.append(TP/len(pred_positive_index))
    return mean_precision

precision_whole = get_precision(x_train, y_train, teacher_model)
precision_clean = get_precision(x_clean, y_clean, teacher_model)
lambda_teacher = sum(precision_clean) / (sum(precision_clean) + sum(precision_whole))
print(precision_whole, precision_clean, lambda_teacher)

file_additional_data = open(dirs + 'file_additional_data.txt', 'a+')
file_additional_data.write('precision of teacher model on whole training dataset' + '\n')
file_additional_data.write(str(precision_whole) + '\n')
file_additional_data.write('precision of teacher model on clean dataset' + '\n')
file_additional_data.write(str(precision_clean) + '\n')
file_additional_data.write('lambda: ' + str(lambda_teacher) + '\n')
file_additional_data.close()

0 3219 <built-in function len>
1 4017 <built-in function len>
2 7463 <built-in function len>
3 4237 <built-in function len>
4 5739 <built-in function len>
5 3538 <built-in function len>
6 7424 <built-in function len>
7 5620 <built-in function len>
8 4847 <built-in function len>
9 3396 <built-in function len>
0 49 <built-in function len>
1 50 <built-in function len>
2 52 <built-in function len>
3 50 <built-in function len>
4 50 <built-in function len>
5 49 <built-in function len>
6 50 <built-in function len>
7 50 <built-in function len>
8 50 <built-in function len>
9 50 <built-in function len>
[0.12488350419384903, 0.14737366193676874, 0.11108133458394748, 0.11682794430021241, 0.12423767206830458, 0.13312605992085924, 0.12082435344827586, 0.1300711743772242, 0.12646998143181348, 0.13457008244994112] [1.0, 1.0, 0.9615384615384616, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] 0.8869677419146537


In [ ]:
# generate a multi-classifier
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
student_model = create_model(architecture, num_classes=10)

file_setting = open(dirs + 'file_setting.txt', 'a+')
file_setting.write('*'*10 + 'architecture of student classifier' + '*'*10 + '\n')
orig_stdout = sys.stdout
sys.stdout = file_setting
print(student_model.summary())
sys.stdout = orig_stdout
file_setting.close()

y_pred = teacher_model.predict(x_train)
y_pseudo = lambda_teacher * y_train + (1-lambda_teacher) * y_pred
History_student = student_model.fit(x_train, y_pseudo, validation_data=(x_test, y_test), batch_size=64, epochs=100, shuffle=True)

file_student.write('training accuracy' + '\n')
file_student.write(str(History_student.history['acc']) + '\n')
file_student.write('test accuracy' + '\n')
file_student.write(str(History_student.history['val_acc']) + '\n')
file_student.close()


Train on 49500 samples, validate on 10000 samples
Epoch 1/100
24640/49500 [=============>................] - ETA: 1:27 - loss: 2.3026 - acc: 0.1011

In [ ]:
# train benchmark
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
benchmark_model = create_model(architecture, num_classes=10)

file_setting = open(dirs + 'file_setting.txt', 'a+')
file_setting.write('*'*10 + 'architecture of benchmark classifier' + '*'*10 + '\n')
orig_stdout = sys.stdout
sys.stdout = file_setting
print(benchmark_model.summary())
sys.stdout = orig_stdout
file_setting.close()

History_benchmark = student_model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=100, shuffle=True)

file_benchmark.write('training accuracy' + '\n')
file_benchmark.write(str(History_benchmark.history['acc']) + '\n')
file_benchmark.write('test accuracy' + '\n')
file_benchmark.write(str(History_benchmark.history['val_acc']) + '\n')
file_benchmark.close()
